In [7]:
# # Load the Drive helper and mount
from google.colab import drive

# # This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

## Preprocessing

In [0]:
train=pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/data/train.csv')
test=pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/data/valid.csv')

In [0]:
train=train.dropna()
test=test.dropna()
train=train.reset_index(drop=True)
test=test.reset_index(drop=True)

In [8]:
train.head()

,context,question,answer_start,text
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary


In [9]:
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [10]:
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()
print(lem.lemmatize("corpora"))

corpus


In [0]:
sw=stopwords.words('english') 
def remove_stopWords(s):
  s=str(s).lower()
  s=s.replace('"', '').replace("'", "").replace("<br />","").replace(",","").replace("(","").replace(")","").replace("-","").replace("/","").replace("!","").replace("?","")
#   s = ' '.join(lem.lemmatize(word) for word in s.split() if word not in sw)
  return s

In [0]:
def get_target(x):
    idx = -1
    for i in range(len(x["context"].split("."))):
        if x["text"] in str(x["context"].split(".")[i]): 
          idx = i
    return idx

In [0]:
test["target"] = test.apply(get_target, axis = 1)

In [0]:
train["target"]=train.apply(get_target,axis=1)

In [0]:
train["pcontext"]=train.context.apply(lambda x: remove_stopWords(x))
train["pquestion"]=train.question.apply(lambda x: remove_stopWords(x))
test["pcontext"]=test.context.apply(lambda x: remove_stopWords(x))
test["pquestion"]=test.question.apply(lambda x: remove_stopWords(x))
test["ptext"]=test.text.apply(lambda x: remove_stopWords(x))
train["ptext"]=train.text.apply(lambda x: remove_stopWords(x))

In [16]:
train.head()

,context,question,answer_start,text,target,pcontext,pquestion,ptext
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,5,architecturally the school has a catholic char...,to whom did the virgin mary allegedly appear i...,saint bernadette soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,2,architecturally the school has a catholic char...,what is in front of the notre dame main building,a copper statue of christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,3,architecturally the school has a catholic char...,the basilica of the sacred heart at notre dame...,the main building
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,4,architecturally the school has a catholic char...,what is the grotto at notre dame,a marian place of prayer and reflection
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,1,architecturally the school has a catholic char...,what sits on top of the main building at notre...,a golden statue of the virgin mary


In [0]:
train.to_csv("/content/drive/My Drive/Colab Notebooks/NLP/data/trainp.csv")
test.to_csv("/content/drive/My Drive/Colab Notebooks/NLP/data/testp.csv")

## Train  a word2vec model

In [0]:
train_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP/data/trainp.csv")
valid_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP/data/testp.csv")

In [0]:
paras = list(train_df["pcontext"].drop_duplicates().reset_index(drop= True))

In [0]:
paras=paras+list(valid_df["pcontext"].drop_duplicates().reset_index(drop= True))

In [0]:
paras_raw = " ".join(paras)

In [23]:
len(paras_raw)

15243030

In [24]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
sentences=paras_raw.split(".")

In [0]:
from nltk.tokenize import word_tokenize 
filtered_sentence=[word_tokenize(sentences[i]) for i in range(len(sentences))]

In [27]:
filtered_sentence[1]

['atop',
 'the',
 'main',
 'buildings',
 'gold',
 'dome',
 'is',
 'a',
 'golden',
 'statue',
 'of',
 'the',
 'virgin',
 'mary']

In [28]:
from gensim.models import Word2Vec
model = Word2Vec(filtered_sentence, min_count=1,seed=123)
print(model)


/usr/local/lib/python3.6/dist-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


Word2Vec(vocab=102348, size=100, alpha=0.025)


In [0]:
model.save('drive/My Drive/Colab Notebooks/NLP/word2vec/word2vec.bin')

## Unsupervised Learning

In [0]:
import pandas as pd
train=pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/data/trainp.csv')
test=pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/data/testp.csv')
train=train.dropna()
test=test.dropna()
train=train.reset_index(drop=True)
test=test.reset_index(drop=True)

In [0]:
from gensim.models import Word2Vec
model = Word2Vec.load('drive/My Drive/Colab Notebooks/NLP/word2vec/word2vec.bin')

In [0]:
def unsupervisedwm(x):
    para=x["pcontext"]
    question=x["pquestion"]
    ans=x["ptext"]
    s=para.split(".")
    min_dist=10000000
    index=-1
    for i in range(len(s)):
      temp=model.wv.wmdistance(s[i].split(" "),question.split(" "))
      if temp<min_dist:
        min_dist=temp
        index=i
    return index
#     print(para)
#     print(question)
#     print(s[index])
#     print(ans)

In [0]:
import numpy as np
from scipy import spatial

def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

def unsupervisedcosine(x):
    para=x["pcontext"]
    question=x["pquestion"]
    ans=x["ptext"]
    s=para.split(".")
#     print(s[0].split(" "),question.split(" "))
    maxsimilar=0
    index=-1
    for i in range(len(s)):
      s1_afv = avg_feature_vector(s[i], model=model, num_features=100, index2word_set=model)
      s2_afv = avg_feature_vector(question, model=model, num_features=100, index2word_set=model)
      sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
      if sim>maxsimilar:
        maxsimilar=sim
        index=i
    return index

In [0]:
train["unsup"] = train.apply(unsupervisedwm, axis = 1)
test["unsup"]= test.apply(unsupervisedwm, axis = 1)

## Accuracy

In [0]:
def accuracy(target, predicted):
    
    acc = (target==predicted).sum()/len(target)
    
    return acc

In [0]:
print("wmdistance accuracy on train set:",accuracy(train["target"],train["unsup"]))

wmdistance accuracy on train set: 0.5008171521960251


In [0]:
print("wmdistance accuracy on test set:",accuracy(test["target"],test["unsup"]))

wmdistance accuracy on test set: 0.5184133295465303


In [0]:
print("cosine similarity accuracy on train set:",accuracy(train["target"],train["unsup"]))

cosine similarity accuracy on train set: 0.4813351903011484


In [0]:
print("cosine similarity accuracy on test set:",accuracy(test["target"],test["unsup"]))

cosine similarity accuracy on test set: 0.47436140018921474
